In [19]:
# Install selenium first and the rest of the needed modules

In [20]:
!pip install requests --quiet
!pip install beautifulsoup4 --quiet
!apt install chromium-chromedriver --quiet


     |████████████████████████████████| 127 kB 4.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.7.2 requires urllib3[socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.
Reading package lists...
Building dependency tree...
Reading state information...
chromium-chromedriver is already the newest version (108.0.5359.71-0ubuntu0.18.04.5).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [28]:
%%bash
sudo apt-get update
sudo pip install selenium

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists...
Looking in indexes: https://pypi.org/simple, https://us

In [22]:
!apt install mongodb > log

In [23]:
# Start mongodb service just in case it is not running
!service mongodb start

 * Starting database mongodb
   ...done.


In [24]:
# Check if mongo service in running successfully
%%bash
ps -ef | grep mongo

mongodb     1581       1  0 12:37 ?        00:00:03 /usr/bin/mongod --config /etc/mongodb.conf
root        2637    2635  0 12:52 ?        00:00:00 grep mongo


In [8]:
# Libraries
import pandas as pd
import requests
import smtplib
import time
import csv
import os

from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
from pymongo import MongoClient
from io import StringIO
from IPython.display import display

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


/usr/local/lib/python3.8/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


## Instantiate Mongodb

In [29]:
mongoClient = MongoClient()
client = MongoClient()

DATABASE_URI="mongodb://localhost:27017"
DATABASE="cryptocurrencies"
CRYPTO_COLLECTION="cryptos"

db = client[DATABASE]

# Create collection if not existing
if "cryptos" not in db.list_collection_names():
  db.create_collection(CRYPTO_COLLECTION)


## Set up the BASE URL and Test it

In [10]:
# Any variables defined in CAPITAL belong to the GLOBAL variables
BASE_URL = 'https://finance.yahoo.com'

response = requests.get(BASE_URL)

if response: print("response.ok : {} , response.status_code : {}".format(response.ok , response.status_code))

response.ok : True , response.status_code : 200


## Set Up the Web Driver
* Ensure the driver is downloaded and within the path

In [26]:

def get_driver(url):
    """Return web driver"""
    chrome_options = Options()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--start-maximized') #
    chrome_options.add_argument('--start-fullscreen')#
    chrome_options.add_argument('--single-process')#
    # serv = Service(os.getcwd()+'/chromedriver')
    driver = webdriver.Chrome('chromedriver', options=chrome_options)
    driver.get(url)
    return driver

## Data Extraction

In [27]:
def get_table_header(driver):
    """Return Table columns in list form """
    header = driver.find_elements(By.TAG_NAME, value= 'th')
    header_list = [item.text for index, item in enumerate(header) if index < 10]
    return header_list

def get_table_rows(driver):
    """Get number of rows available on the page """
    tablerows = len(driver.find_elements(By.XPATH, value='//*[@id="scr-res-table"]/div[1]/table/tbody/tr'))
    return tablerows  

## Data Cleaning

In [13]:
def parse_table_rows(rownum, driver, header_list):
    """get the data for one row at a time and return column value in the form of dictionary"""
    row_dictionary = {}
    #time.sleep(1/3)
    for index , item in enumerate(header_list):
        time.sleep(1/20)
        column_xpath = '//*[@id="scr-res-table"]/div[1]/table/tbody/tr[{}]/td[{}]'.format(rownum, index+1)
        row_dictionary[item] = driver.find_element(By.XPATH, value=column_xpath).text
    return row_dictionary

def parse_multiple_pages(driver, total_crypto):
    """Loop through each row, perform Next button click at the end of page 
    return total_crypto numbers of rows 
    """
    table_data = []
    page_num = 1
    is_scraping = True
    header_list = get_table_header(driver)

    while is_scraping:
        table_rows = get_table_rows(driver)
        print('Found {} rows on Page : {}'.format(table_rows, page_num))
        print('Parsing Page : {}'.format(page_num))
        table_data += [parse_table_rows(i, driver, header_list) for i in range (1, table_rows + 1)]
        total_count = len(table_data)
        print('Total rows scraped : {}'.format(total_count))
        if total_count >= total_crypto:
            print('Done Parsing..')
            is_scraping = False
        else:    
            print('Clicking Next Button')
            element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="scr-res-table"]/div[2]/button[3]')))
            element.click() 
            page_num += 1
    return table_data


## Ingesting Data to Storage, i.e MongoDB

In [14]:
def save_data_tomongo(dataframe):
    # Saves dataframe to mongodb
    db.cryptos.insert_many(dataframe.to_dict(orient="records"))
    print(f"Successfully saved cleaned data to Database!")

## The Factory Code

In [31]:



def scrape_yahoo_crypto(url, total_crypto, path=None):
    """Get the list of yahoo finance crypto-currencies and write them to CSV file """
    if path is None:
        path = 'crypto-currencies.csv'
    print('Creating driver')
    driver = get_driver(url)    
    table_data = parse_multiple_pages(driver, total_crypto)
    driver.close()
    driver.quit()
    print('Save the data to a CSV')
    table_df = pd.DataFrame(table_data)
    table_df.to_csv(path, index=None)
    display(table_df.head())
    print("Save to mongodb")
    save_data_tomongo(table_df)
    return table_df 

if __name__ == "__main__" :
    
    YAHOO_FINANCE_URL = BASE_URL+'/crypto'
    TOTAL_CRYPTO = 50
    crypto_df = scrape_yahoo_crypto(YAHOO_FINANCE_URL, TOTAL_CRYPTO,'crypto-currencies.csv')

print("Processing Done")

Creating driver
Found 25 rows on Page : 1
Parsing Page : 1
Total rows scraped : 25
Clicking Next Button
Found 25 rows on Page : 2
Parsing Page : 2
Total rows scraped : 50
Done Parsing..
Save the data to a CSV


,Symbol,Name,Price (Intraday),Change,% Change,Market Cap,Volume in Currency (Since 0:00 UTC),Volume in Currency (24Hr),Total Volume All Currencies (24Hr),Circulating Supply
0,BTC-USD,Bitcoin USD,"16,761.07",+21.06,+0.13%,322.449B,12.79B,12.79B,12.79B,19.238M
1,ETH-USD,Ethereum USD,"1,187.58",+4.52,+0.38%,145.328B,3.927B,3.927B,3.927B,122.374M
2,USDT-USD,Tether USD,1.0001,+0.0000,+0.00%,66.166B,16.562B,16.562B,16.562B,66.157B
3,USDC-USD,USD Coin USD,1.0001,-0.0001,-0.01%,45.074B,1.895B,1.895B,1.895B,45.07B
4,BNB-USD,BNB USD,250.20,+3.06,+1.24%,40.024B,729.338M,729.338M,729.338M,159.967M


Save to mongodb
Successfully saved cleaned data to Database!
Processing Done


In [16]:
def check_successful_dataingestion():
  for entry in db.cryptos.find():
    print(entry)


In [17]:

check_successful_dataingestion()

{'_id': ObjectId('63a05b2d3e38f489a7240e8a'), 'Symbol': 'BTC-USD', 'Name': 'Bitcoin USD', 'Price (Intraday)': '16,739.45', 'Change': '+14.24', '% Change': '+0.09%', 'Market Cap': '322.033B', 'Volume in Currency (Since 0:00 UTC)': '12.693B', 'Volume in Currency (24Hr)': '12.693B', 'Total Volume All Currencies (24Hr)': '12.693B', 'Circulating Supply': '19.238M'}
{'_id': ObjectId('63a05b2d3e38f489a7240e8b'), 'Symbol': 'ETH-USD', 'Name': 'Ethereum USD', 'Price (Intraday)': '1,184.23', 'Change': '+1.48', '% Change': '+0.13%', 'Market Cap': '144.919B', 'Volume in Currency (Since 0:00 UTC)': '3.896B', 'Volume in Currency (24Hr)': '3.896B', 'Total Volume All Currencies (24Hr)': '3.896B', 'Circulating Supply': '122.374M'}
{'_id': ObjectId('63a05b2d3e38f489a7240e8c'), 'Symbol': 'USDT-USD', 'Name': 'Tether USD', 'Price (Intraday)': '1.0001', 'Change': '+0.0000', '% Change': '+0.00%', 'Market Cap': '66.167B', 'Volume in Currency (Since 0:00 UTC)': '16.337B', 'Volume in Currency (24Hr)': '16.337B',

In [18]:
# Shutdown mongodb, don't run this until you are done otherwise the db instance will be closed
!mongod --shutdown

There doesn't seem to be a server running with dbpath: /data/db
